In [ ]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
import ROOT
import matplotlib.pyplot as plt
import get_truth_neutron_data
import analysis as a
from matplotlib import rc
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
rc('text', usetex=False)

In [ ]:
truth = get_truth_neutron_data.extract() #truth neutrons
a = a.analysis(E_cut = {'palila': 9.0, 'iiwi': 8.8, 'tako': 4.6, 'nene': 5.6, 'elepaio': 6.4, 'humu': 5.6}, recoils_only = True, fei4_restrict = True) #data and digitized recoils

In [ ]:
MC = truth.get_MC_data('RBB_Lumi')
cuts = truth.apply_RBB_cuts()
MC_LER = truth.get_MC_data('Touschek_LER')
MC_HER = truth.get_MC_data('Touschek_HER')
datamc = a.compute_data_MC_ratios("Cont_inj", E_cut = {'palila': 9.0, 'iiwi': 8.8, 'tako': 4.6, 'nene': 5.6, 'elepaio': 6.4, 'humu': 5.6})[2]
datamc_storage = a.compute_data_MC_ratios("Decay", E_cut = {'palila': 9.0, 'iiwi': 8.8, 'tako': 4.6, 'nene': 5.6, 'elepaio': 6.4, 'humu': 5.6})[2]

In [ ]:
datamc-datamc_storage

In [ ]:
datamc

In [ ]:
iiwi0 = MC['iiwi']

nene0 = MC['nene']

humu0 = MC['humu']

tako0 = MC['tako']

palila0 = MC['palila']

elepaio0 = MC['elepaio']


iiwi = cuts['iiwi']

nene = cuts['nene']

humu = cuts['humu']

tako = cuts['tako']

palila = cuts['palila']

elepaio = cuts['elepaio']

iiwi_SB = MC_LER['iiwi']
nene_SB = MC_LER['nene']
humu_SB = MC_LER['humu']

palila_SB = MC_HER['palila']
tako_SB = MC_HER['tako']
elepaio_SB = MC_HER['elepaio']

### Convert to 1MeV fluxes weighted by NIEL

In [ ]:
def add_NIEL_weights(df, ekey = 'truthNeutronEnergy'):
    conversion = [] #list to be added to dataframe
    e = df[ekey].to_numpy() #use numpy to speed things up
    neutrons = pd.read_csv('neutrons.csv')
    neutrons = neutrons.drop(columns = neutrons.columns[0])
    conv = neutrons[['E','weight']].to_numpy()
    for i in range(0,len(e)):
        index = 0
        diff = 1e9
        for j in range(0,len(conv)):
            if np.abs(e[i] - conv[j][0]) < diff:
                diff = np.abs(e[i] - conv[j][0])
                index = j
        conversion.append(conv[index][1])
    df['NIEL_weight'] = conversion
add_NIEL_weights(tako)
add_NIEL_weights(palila)
add_NIEL_weights(elepaio)
add_NIEL_weights(iiwi)
add_NIEL_weights(nene)
add_NIEL_weights(humu)
add_NIEL_weights(tako0)
add_NIEL_weights(palila0)
add_NIEL_weights(elepaio0)
add_NIEL_weights(iiwi0)
add_NIEL_weights(nene0)
add_NIEL_weights(humu0)
add_NIEL_weights(tako_SB)
add_NIEL_weights(palila_SB)
add_NIEL_weights(elepaio_SB)
add_NIEL_weights(iiwi_SB)
add_NIEL_weights(nene_SB)
add_NIEL_weights(humu_SB)

In [ ]:
print(len(tako), tako['NIEL_weight'].sum())
print(len(palila), palila['NIEL_weight'].sum())
print(len(elepaio), elepaio['NIEL_weight'].sum())
print(len(iiwi), iiwi['NIEL_weight'].sum())
print(len(nene), nene['NIEL_weight'].sum())
print(len(humu), humu['NIEL_weight'].sum())
print()
print(len(tako0), tako0['NIEL_weight'].sum())
print(len(palila0), palila0['NIEL_weight'].sum())
print(len(elepaio0), elepaio0['NIEL_weight'].sum())
print(len(iiwi0), iiwi0['NIEL_weight'].sum())
print(len(nene0), nene0['NIEL_weight'].sum())
print(len(humu0), humu0['NIEL_weight'].sum())
print()
print(len(tako_SB), tako_SB['NIEL_weight'].sum())
print(len(palila_SB), palila_SB['NIEL_weight'].sum())
print(len(elepaio_SB), elepaio_SB['NIEL_weight'].sum())
print(len(iiwi_SB), iiwi_SB['NIEL_weight'].sum())
print(len(nene_SB), nene_SB['NIEL_weight'].sum())
print(len(humu_SB), humu_SB['NIEL_weight'].sum())
print()
print(len(tako)/len(tako0), tako['NIEL_weight'].sum()/tako0['NIEL_weight'].sum())
print(len(palila)/len(palila0), palila['NIEL_weight'].sum()/palila0['NIEL_weight'].sum())
print(len(elepaio)/len(elepaio0), elepaio['NIEL_weight'].sum()/elepaio0['NIEL_weight'].sum())
print(len(iiwi)/len(iiwi0), iiwi['NIEL_weight'].sum()/iiwi0['NIEL_weight'].sum())
print(len(nene)/len(nene0), nene['NIEL_weight'].sum()/nene0['NIEL_weight'].sum())
print(len(humu)/len(humu0), humu['NIEL_weight'].sum()/humu0['NIEL_weight'].sum())

### Compute estimated fluxes

In [ ]:
### Need to use the appropriate column of data_MC

bgType = 'RBB_Lumi'


def estimate_fluxes(bgType,count,tpc): #Touschek_LER, Touschek_HER, or RBB_Lumi. Count is number of V0s from hotspot
    area = 11.3*2.95 #cm^2
    if bgType == 'Touschek_LER':
        time = 0.4 #scale parameters down
        col = 'LER_T'
    elif bgType == 'Touschek_HER':
        time = 1.6
        col = 'HER_T'
    else:
        time = 2.2e-3 * 25 #25 is to scale down to L = 1e34
        col = 'Lumi'
    flux = count*datamc.loc[datamc.index==tpc][col]/(area*time)
    flux_err = count*datamc.loc[datamc.index==tpc][col+'_err']/(area*time)
    return flux[0], flux_err[0]

#NIEL WEIGHTED
iiwi_flux_weighted, iiwi_flux_err_weighted = estimate_fluxes(bgType,iiwi0['NIEL_weight'].sum(),'iiwi')
nene_flux_weighted, nene_flux_err_weighted = estimate_fluxes(bgType,nene0['NIEL_weight'].sum(),'nene')
humu_flux_weighted, humu_flux_err_weighted = estimate_fluxes(bgType,humu0['NIEL_weight'].sum(),'humu')
tako_flux_weighted, tako_flux_err_weighted = estimate_fluxes(bgType,tako0['NIEL_weight'].sum(),'tako')
palila_flux_weighted, palila_flux_err_weighted = estimate_fluxes(bgType,palila0['NIEL_weight'].sum(),'palila')
elepaio_flux_weighted, elepaio_flux_err_weighted = estimate_fluxes(bgType,elepaio0['NIEL_weight'].sum(),'elepaio')
#RAW FLUX
iiwi_flux, iiwi_flux_err = estimate_fluxes(bgType,len(iiwi0),'iiwi')
nene_flux, nene_flux_err = estimate_fluxes(bgType,len(nene0),'nene')
humu_flux, humu_flux_err = estimate_fluxes(bgType,len(humu0),'humu')
tako_flux, tako_flux_err = estimate_fluxes(bgType,len(tako0),'tako')
palila_flux, palila_flux_err = estimate_fluxes(bgType,len(palila0),'palila')
elepaio_flux, elepaio_flux_err = estimate_fluxes(bgType,len(elepaio0),'elepaio')

print(bgType)
print('Weighted Fluxes: iiwi = %s +/- %s'%(iiwi_flux_weighted, iiwi_flux_err_weighted))
print('Weighted Fluxes: nene = %s +/- %s'%(nene_flux_weighted, nene_flux_err_weighted))
print('Weighted Fluxes: humu = %s +/- %s'%(humu_flux_weighted, humu_flux_err_weighted))
print('Weighted Fluxes: palila = %s +/- %s'%(palila_flux_weighted, palila_flux_err_weighted))
print('Weighted Fluxes: tako = %s +/- %s'%(tako_flux_weighted, tako_flux_err_weighted))
print('Weighted Fluxes: elepaio = %s +/- %s'%(elepaio_flux_weighted, elepaio_flux_err_weighted))

print('RAW Fluxes: iiwi = %s +/- %s'%(iiwi_flux, iiwi_flux_err))
print('RAW Fluxes: nene = %s +/- %s'%(nene_flux, nene_flux_err))
print('RAW Fluxes: humu = %s +/- %s'%(humu_flux, humu_flux_err))
print('RAW Fluxes: palila = %s +/- %s'%(palila_flux, palila_flux_err))
print('RAW Fluxes: tako = %s +/- %s'%(tako_flux, tako_flux_err))
print('RAW Fluxes: elepaio = %s +/- %s'%(elepaio_flux, elepaio_flux_err))


In [ ]:
### Need to use the appropriate column of data_MC
def estimate_fluxes(bgType,count,tpc): #Touschek_LER, Touschek_HER, or RBB_Lumi. Count is number of V0s from hotspot
    area = 11.3*2.95 #cm^2
    if bgType == 'Touschek_LER':
        time = 0.4 #scale parameters down
        col = 'LER_T'
    elif bgType == 'Touschek_HER':
        time = 1.6
        col = 'HER_T'
    else:
        time = 2.2e-3 * 25 #25 is to scale down to L = 1e34
        col = 'Lumi'
    flux = count*datamc.loc[datamc.index==tpc][col]/(area*time)
    flux_err = count*datamc.loc[datamc.index==tpc][col+'_err']/(area*time)
    return flux[0], flux_err[0]

#NIEL WEIGHTED
iiwi_flux_weighted, iiwi_flux_err_weighted = estimate_fluxes('Touschek_LER',iiwi_SB['NIEL_weight'].sum(),'iiwi')
nene_flux_weighted, nene_flux_err_weighted = estimate_fluxes('Touschek_LER',nene_SB['NIEL_weight'].sum(),'nene')
humu_flux_weighted, humu_flux_err_weighted = estimate_fluxes('Touschek_LER',humu_SB['NIEL_weight'].sum(),'humu')
tako_flux_weighted, tako_flux_err_weighted = estimate_fluxes('Touschek_HER',tako_SB['NIEL_weight'].sum(),'tako')
palila_flux_weighted, palila_flux_err_weighted = estimate_fluxes('Touschek_HER',palila_SB['NIEL_weight'].sum(),'palila')
elepaio_flux_weighted, elepaio_flux_err_weighted = estimate_fluxes('Touschek_HER',elepaio_SB['NIEL_weight'].sum(),'elepaio')
#RAW FLUX
iiwi_flux, iiwi_flux_err = estimate_fluxes('Touschek_LER',len(iiwi_SB),'iiwi')
nene_flux, nene_flux_err = estimate_fluxes('Touschek_LER',len(nene_SB),'nene')
humu_flux, humu_flux_err = estimate_fluxes('Touschek_LER',len(humu_SB),'humu')
tako_flux, tako_flux_err = estimate_fluxes('Touschek_HER',len(tako_SB),'tako')
palila_flux, palila_flux_err = estimate_fluxes('Touschek_HER',len(palila_SB),'palila')
elepaio_flux, elepaio_flux_err = estimate_fluxes('Touschek_HER',len(elepaio_SB),'elepaio')

print(bgType)
print('Weighted Fluxes: iiwi = %s +/- %s'%(iiwi_flux_weighted, iiwi_flux_err_weighted))
print('Weighted Fluxes: nene = %s +/- %s'%(nene_flux_weighted, nene_flux_err_weighted))
print('Weighted Fluxes: humu = %s +/- %s'%(humu_flux_weighted, humu_flux_err_weighted))
print('Weighted Fluxes: palila = %s +/- %s'%(palila_flux_weighted, palila_flux_err_weighted))
print('Weighted Fluxes: tako = %s +/- %s'%(tako_flux_weighted, tako_flux_err_weighted))
print('Weighted Fluxes: elepaio = %s +/- %s'%(elepaio_flux_weighted, elepaio_flux_err_weighted))

print('RAW Fluxes: iiwi = %s +/- %s'%(iiwi_flux, iiwi_flux_err))
print('RAW Fluxes: nene = %s +/- %s'%(nene_flux, nene_flux_err))
print('RAW Fluxes: humu = %s +/- %s'%(humu_flux, humu_flux_err))
print('RAW Fluxes: palila = %s +/- %s'%(palila_flux, palila_flux_err))
print('RAW Fluxes: tako = %s +/- %s'%(tako_flux, tako_flux_err))
print('RAW Fluxes: elepaio = %s +/- %s'%(elepaio_flux, elepaio_flux_err))


In [ ]:

lw=2

plt.rc('legend', fontsize=18)
plt.rc('xtick', labelsize=24)
plt.rc('ytick', labelsize=24)
plt.rc('axes', labelsize=26)
plt.rc('axes', titlesize=26)

fig, ax = plt.subplots(1,1, figsize = (8,6))

ax.set_xlabel('Neutron Energy [keV]')
#ax.set_ylabel(r'Est. Flux [Hz$\cdot$cm$^{-2}$]')
ax.set_ylabel(r'Normalized Counts')
ax.set_ylim(0,0.1)

(counts, bins) = np.histogram(palila_SB['truthNeutronEnergy'], bins=100, range = (0,1e4))
#factor = iiwi_flux_weighted/iiwi['NIEL_weight'].sum()
factor = 1/counts.sum()
ax.hist(bins[:-1], bins, weights=factor*counts, histtype = 'step', label = 'z=-5.6m', lw=lw)

(counts, bins) = np.histogram(elepaio_SB['truthNeutronEnergy'], bins=100, range = (0,1e4))
#factor = nene_flux_weighted/nene['NIEL_weight'].sum()
factor = 1/counts.sum()
ax.hist(bins[:-1], bins, weights=factor*counts, histtype = 'step', label = 'z=-8.0m', lw=lw)

(counts, bins) = np.histogram(tako_SB['truthNeutronEnergy'], bins=100, range = (0,1e4))
#factor = humu_flux_weighted/humu['NIEL_weight'].sum()
factor = 1/counts.sum()
ax.hist(bins[:-1], bins, weights=factor*counts, histtype = 'step', label = 'z=-14m', lw=lw)


(counts, bins) = np.histogram(iiwi_SB['truthNeutronEnergy'], bins=100, range = (0,1e4))
#factor = iiwi_flux_weighted/iiwi['NIEL_weight'].sum()
factor = 1/counts.sum()
ax.hist(bins[:-1], bins, weights=factor*counts, histtype = 'step', label = 'z=+6.6m', lw=lw, color = 'tab:red')

(counts, bins) = np.histogram(nene_SB['truthNeutronEnergy'], bins=100, range = (0,1e4))
#factor = nene_flux_weighted/nene['NIEL_weight'].sum()
factor = 1/counts.sum()
ax.hist(bins[:-1], bins, weights=factor*counts, histtype = 'step', label = 'z=+14m', lw=lw, color = 'tab:purple')

(counts, bins) = np.histogram(humu_SB['truthNeutronEnergy'], bins=100, range = (0,1e4))
#factor = humu_flux_weighted/humu['NIEL_weight'].sum()
factor = 1/counts.sum()
ax.hist(bins[:-1], bins, weights=factor*counts, histtype = 'step', label = 'z=+16m', lw=lw, color = 'tab:brown')

ax.grid()
plt.legend()

plt.tight_layout()
plt.savefig('truth_single_beam_touschek_spectrum.jpg')
plt.show()


'''
lw=2

plt.rc('legend', fontsize=18)
plt.rc('xtick', labelsize=24)
plt.rc('ytick', labelsize=24)
plt.rc('axes', labelsize=26)
plt.rc('axes', titlesize=26)

fig, ax = plt.subplots(1,1, figsize = (8,6))

ax.set_xlabel('Neutron Energy [keV]')
ax.set_ylabel(r'Est. Flux [Hz$\cdot$cm$^{-2}$]')
ax.set_ylim(0,250)

(counts, bins) = np.histogram(palila['truthNeutronEnergy'], bins=100, range = (0,1e4))
factor = palila_flux_weighted/palila['NIEL_weight'].sum()
ax.hist(bins[:-1], bins, weights=factor*counts, histtype = 'step', label = 'z=-5.6m', lw=lw)

(counts, bins) = np.histogram(tako['truthNeutronEnergy'], bins=100, range = (0,1e4))
factor = tako_flux_weighted/tako['NIEL_weight'].sum()
ax.hist(bins[:-1], bins, weights=factor*counts, histtype = 'step', label = 'z=-8.0m', lw=lw)

(counts, bins) = np.histogram(elepaio['truthNeutronEnergy'], bins=100, range = (0,1e4))
factor = elepaio_flux_weighted/elepaio['NIEL_weight'].sum()
ax.hist(bins[:-1], bins, weights=factor*counts, histtype = 'step', label = 'z=-14m', lw=lw)

ax.grid()
plt.legend()

plt.tight_layout()
plt.savefig('truth_HER_touschek_spectrum.jpg')
plt.show()
'''

In [ ]:
palila['NIEL_weight']

In [ ]:
lw=2

plt.rc('legend', fontsize=18)
plt.rc('xtick', labelsize=24)
plt.rc('ytick', labelsize=24)
plt.rc('axes', labelsize=26)
plt.rc('axes', titlesize=26)

fig, ax = plt.subplots(1,1, figsize = (8,6))
(counts, bins) = np.histogram(palila0['truthNeutronEnergy'], bins=100, range = (0,1e4))
#factor = palila_flux_weighted/palila['NIEL_weight'].sum()/12.5
factor = 1/counts.sum()
ax.hist(bins[:-1], bins, weights=factor*counts, histtype = 'step', label = 'z=-5.6m', lw=lw)
ax.set_xlabel('Neutron Energy [keV]')
#ax.set_ylabel(r'Est. Flux [Hz$\cdot$cm$^{-2}$]')
ax.set_ylabel(r'Normalized Counts')
ax.set_ylim(0,0.1)

(counts, bins) = np.histogram(tako0['truthNeutronEnergy'], bins=100, range = (0,1e4))
#factor = tako_flux_weighted/tako['NIEL_weight'].sum()/12.5
factor = 1/counts.sum()
ax.hist(bins[:-1], bins, weights=factor*counts, histtype = 'step', label = 'z=-8.0m', lw=lw)

(counts, bins) = np.histogram(elepaio0['truthNeutronEnergy'], bins=100, range = (0,1e4))
#factor = elepaio_flux_weighted/elepaio['NIEL_weight'].sum()/12.5
factor = 1/counts.sum()
ax.hist(bins[:-1], bins, weights=factor*counts, histtype = 'step', label = 'z=-14m', lw=lw)

(counts, bins) = np.histogram(iiwi0['truthNeutronEnergy'], bins=100, range = (0,1e4))
#factor = iiwi_flux_weighted/iiwi['NIEL_weight'].sum()/12.5
factor = 1/counts.sum()
ax.hist(bins[:-1], bins, weights=factor*counts, histtype = 'step', label = 'z=+6.6m', lw=lw, color = 'tab:red')

(counts, bins) = np.histogram(nene0['truthNeutronEnergy'], bins=100, range = (0,1e4))
#factor = nene_flux_weighted/nene['NIEL_weight'].sum()/12.5
factor = 1/counts.sum()
ax.hist(bins[:-1], bins, weights=factor*counts, histtype = 'step', label = 'z=+14m', lw=lw, color = 'tab:purple')

(counts, bins) = np.histogram(humu0['truthNeutronEnergy'], bins=100, range = (0,1e4))
#factor = humu_flux_weighted/humu['NIEL_weight'].sum()/12.5
factor = 1/counts.sum()
ax.hist(bins[:-1], bins, weights=factor*counts, histtype = 'step', label = 'z=+16m', lw=lw, color = 'tab:brown')

ax.grid()
plt.legend()

plt.tight_layout()
plt.savefig('truth_lumi_spectrum.jpg')
plt.show()

### Compare Belle detector neutron rates

In [ ]:
import os
import uproot as ur

def compare_fractions_of_events(bgType, detector, base_path = '/home/jeef/data/phase3/Belle2_neutron_ntuples/'):
    detector = detector.upper()
    bgType = bgType.upper()
    fnames = {'BHWIDELARGEANGLE':'BHWideLargeAngle_Lumi_all.root',
             'BHWIDE':'BHWide_Lumi_all.root',
             'RBB':'RBB_Lumi_all.root',
             'TWOPHOTON':'twoPhoton_Lumi_all.root'}
    f = fnames[bgType]
    detID = {'IR':0, 'PXD':1, 'SVD':2, 'CDC':3, 'ARICH':4, 'TOP':5, 'ECL':6, 'EKLM':7, 'BKLM':8}    
    farbeamline_path = base_path + 'farbeamline/'
    nofarbeamline_path = base_path + 'no_farbeamline/'
    fbl = ur.open(farbeamline_path + f)['tree'].pandas.df()
    nfbl = ur.open(nofarbeamline_path + f)['tree'].pandas.df()
    neutrons_inside = nfbl.loc[nfbl['subDet'] == detID[detector]]['neutronWeight'].to_numpy().sum()
    neutrons_outside = fbl.loc[fbl['subDet'] == detID[detector]]['neutronWeight'].to_numpy().sum()
    diff = neutrons_outside-neutrons_inside
    frac = diff/(neutrons_outside)
    print(neutrons_inside)
    print(neutrons_outside)
    print(frac)
    return frac
compare_fractions_of_events('BHWIDE', 'ARICH')